<img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/>

# 构建 DeepStream 应用 #
在掌握基础知识后，即可准备构建基本的视频 AI 应用。得益于 [gst-python](https://gstreamer.freedesktop.org/modules/gst-python.html)（GStreamer 框架的 Python 绑定），DeepStream 支持 C/C++ 和 Python 的应用开发。为帮助您更轻松地开始使用，DeepStream 附带了几个 C/C++ 和 Python 参考应用。您可以在 [NVIDIA-AI-IOT](https://github.com/NVIDIA-AI-IOT/deepstream_reference_apps) Github 页面中找到这些示例应用。在本实验中，我们将使用 Python。其他开发方法包括：
* 使用源代码构建应用
* 使用命令行构建应用
* 使用配置文件构建应用

**注意**：安装的最新 DeepStream SDK 中附带单独的 [Python 绑定](https://docs.nvidia.com/metropolis/deepstream/python-api/index.html)，可支持访问 DeepStream 元数据。

## 学习目标 ##
在此 Notebook 中，您将学习如何使用 Python 构建 DeepStream 工作流水线，内容包括：
* 规划工作流水线的架构
* 使用规范文件配置深度学习推理
* 处理元数据
* 测量工作流水线性能

**目录**
<br>
本 Notebook 涵盖以下部分：
1. [设计工作流水线](#s1)
    * [预览输入视频](#s1.1)
2. [深度学习模型准备工作](#s2)
    * [TrafficCamNet 物体检测模型](#s2.1)
    * [聚类](#s2.2)
    * [安装 NGC CLI](#s2.3)
2. [构建视频 AI 应用 - 第一部分](#s3)
    * [工作流水线的组件](#s3.1)
    * [初始化 GStreamer](#s3.2)
    * [创建工作流水线的元件](#s3.3)
    * [练习 #1 - 了解 NvInfer 配置文件](#e1)
    * [关联工作流水线的元件](#s3.4)
    * [探查](#s3.5)
    * [启动工作流水线](#s3.6)
    * [汇总视频数据](#s3.7)
7. [性能分析](#s4)
    * [吞吐量](#s4.1)
    * [延迟](#s4.2)
    * [利用率](#s4.3)
3. [构建视频 AI 应用 - 第二部分](#s5)
    * [工作流水线的组件](#s5.1)
    * [初始化 GStreamer](#s5.2)
    * [创建工作流水线的元件](#s5.3)
    * [关联工作流水线的元件](#s5.4)
    * [探查](#s5.5)
    * [启动工作流水线](#s5.6)
    * [查看推理](#s5.7)

<a name='s1'></a>
## 设计工作流水线 ##
构建视频 AI 应用，首先需要根据用例设计项目。在此实操练习中，我们将构建简单的 DeepStream 工作流水线，以根据停车场摄像头输入画面准确检测汽车。此工作流水线与上一个 Notebook 中示例应用的工作流水线相同。其中的步骤概述如下：
1. 读取视频文件中的数据
2. 对各帧执行深度学习推理
3. 使用推理结果开展后期处理
4. 将数据写入文件

<p><img src='images/steps.png' width=720></p>

为帮助您学习，我们有策略地将整个工作流水线分成两部分。
* 在第一部分，我们将构建一个简单的 DeepStream 工作流水线，以使用视频数据，并对各帧执行推理，旨在统计视频中出现过的物体数量。
* 在第二部分，我们将在 DeepStream 工作流中添加绘制边界框和文本标签，以及将输出写入文件的功能。

<a name='s1.1'></a>
### 预览输入视频 ###
先看一下原始输入视频，然后使用 `ffprobe` 命令行实用程序来了解其格式（[按需参阅文档](https://ffmpeg.org/ffprobe.html)）。使用 `ffprobe` 命令时，`-i` 选项可用于读取输入 [URL](https://en.wikipedia.org/wiki/URL)，而 `-hide_banner` 选项可用于阻止打印横幅以减少输出。

In [ ]:
# DO NOT CHANGE THIS CELL
import os

# Set the input video path to an environment variable
os.environ['TARGET_VIDEO_PATH']='data/sample_30.h264'
os.environ['TARGET_VIDEO_PATH_MP4']='sample_30.mp4'

target_video_path=os.environ['TARGET_VIDEO_PATH']
target_video_path_mp4=os.environ['TARGET_VIDEO_PATH_MP4']

# Analyze video
!ffprobe -i $TARGET_VIDEO_PATH \
         -hide_banner

H.264 编码视频文件不能直接在 JupyterLab 中进行播放。若要在 JupyterLab 中播放视频，需要先将视频数据转换为容器文件。[FFmpeg](https://ffmpeg.org/) 工具是一款超快速的视频和音频转换器，使用通用语法：<br> `ffmpeg [global_options] {[input_file_options] -i input_url} ... {[output_file_options] output_url} ...` <br> 使用 `ffmpeg` 命令时，`-i` 选项可用于读取输入 URL，`-loglevel quiet` 选项可用于屏蔽日志以减少输出，而 `-y` 标志可用于覆盖所有现有同名输出文件。

In [ ]:
# DO NOT CHANGE THIS CELL
from IPython.display import Video

# Convert the H.264 encoded video file to MP4 container file - this will generate the sample_30.mp4 file
!ffmpeg -i $TARGET_VIDEO_PATH $TARGET_VIDEO_PATH_MP4 \
        -y \
        -loglevel quiet

# View the input video
Video(target_video_path_mp4, width=720)

<p><img src='images/important.png' width=720></p>

请注意视频帧的**维度**、**帧速率**、**编解码器**、**色彩模型**和**文件格式**。在本例中，文件由 **H.264** 编解码器完成压缩。该视频像素宽高比为 **882** x **692** 的帧组成，并且采用 **YUV420p** 色彩编码系统，其帧速率为 **30 FPS**（帧/秒）。

<a name='s2'></a>
## 深度学习模型准备工作 ##
对于深度学习推理，我们将使用 [**N**VIDIA **G**PU **C**loud](https://ngc.nvidia.com/)（GPU 优化型 AI 和高性能计算 (HPC) 软件中心）提供的预训练模型。NGC 为大量 AI 任务提供了预训练模型。我们将使用 [NGC](https://catalog.ngc.nvidia.com/) 中名为 [TrafficCamNet](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/tao/models/trafficcamnet) 的专用模型。

<a name='s2.1'></a>
### TrafficCamNet 物体检测模型 ###
据[模型卡](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/tao/models/trafficcamnet)表明，TrafficCamNet 物体检测模型可*在图像中* 检测出一个或多个涉及四种类别（汽车、行人、路标和两轮车）的物理物体，并用方框突出显示每个物体，同时为其提供一个类别标签。

例如，提供输入图像/帧后，推理引擎会生成边界框坐标以及类别标签：
<p><img src='images/sample_inference.png' width=720></p>

然后，我们可以*有选择地* 执行后期处理，并在原始帧上绘制边界框和文本标签。
<p><img src='images/sample_bounding_boxes.png' width=720></p>

<a name='s2.2'></a>
### 聚类 ###
此插件用于定义与应用相关的深度学习任务。张量数据是经过推理得出的原始张量输出。如果要尝试检测物体，则需要通过解析和聚类算法对这些张量数据执行后期处理，以便在检测到的物体周围创建边界框。在生成张量数据后，您可以在 DeepStream 应用中自定义解析算法。此操作很重要，因为张量数据的大小和维度以及解析数据所需的解析和聚类算法将视 AI 模型的类型而定。如果想将自定义模型或新的后期处理技术引入 DeepStream，您会发现学习与元数据的交互方式非常有用。
<p><img src='images/no_clustering.png'></p>

<a name='s2.3'></a>
### 安装 NGC CLI ###
下方单元格将安装 [NGC CLI](https://ngc.nvidia.com/setup/installers/cli)，并从 NGC 中下载素材。

若要从注册服务器中下载模型，请使用 `ngc registry model download-version <org-name>/<model-name:version>` 命令，同时指定模型名称与版本。系统会将模型下载到与当前目录中的模型名称对应的文件夹中，或者您也可以使用 `-d` 或 `--dest` 选项指定下载位置。

此处已预先下载好所需的模型，运行下方单元格以解压。

In [ ]:
# DO NOT CHANGE THIS CELL
!tar -xvf ngc_assets/trafficcamnet_vpruned_v1.0.tar -C ngc_assets/

<a name='s3'></a>
## 构建视频 AI 应用 - 第一部分 ##
在第一部分，我们将构建一个简单的 DeepStream 工作流水线，以使用视频数据，并对各帧进行推理，旨在统计视频中出现过的汽车的数量。

<a name='s3.1'></a>
### 工作流水线组件 ###
这是应用的工作流水线架构。我们已根据每个插件的功能选择插件。
<p><img src="images/deepstream_pipeline_pt1.png" width='720'></p>

我们来了解下各个插件：

* [filesrc](https://gstreamer.freedesktop.org/documentation/coreelements/filesrc.html) 插件可读取本地文件中的数据。此外，还有其他可用的数据读取插件，这些插件可读取摄像头、RTSP 流和磁盘等各类源中的数据。
* [h264parse](https://gstreamer.freedesktop.org/documentation/videoparsersbad/h264parse.html) 插件用于将输入的基本 H.264 流解析为帧大小（以字节为单位）的数据。
* [nvv4l2decoder](https://gist.github.com/nins-k/6f7377629000894deedc7f14d7ef64fe) 可使用适当的编解码器（在本例中为 H.264）对输入数据进行解码。
* [nvstreammux](https://docs.nvidia.com/metropolis/deepstream/dev-guide/text/DS_plugin_gst-nvstreammux.html) 插件是必要的流复用器。此视频复用器有助于接收 `n` 个输入流，并将其转换为连续的批量帧。即使我们的简单应用只处理一个输入流，此插件也需要作为 `nvinfer` 接收已附加 `NvDsBatchMeta` 结构的*批量* 缓冲区。
    * 此插件会用到 `width`、`height` 和 `batch-size` 参数。
* [nvinfer](https://docs.nvidia.com/metropolis/deepstream/dev-guide/text/DS_plugin_gst-nvinfer.html) 插件可根据网络要求转换输入帧（格式转换和缩放），并将经转换的数据传递到低层级库。低层级库会预处理经过转换的帧（进行归一化和均值减法处理），并生成最终浮点 RGB/BGR/GRAY 平面数据，系统会将这些数据传递至 TensorRT 引擎进行推理。低层级库所生成的输出类型视网络类型而定。我们可为 `TrafficCamNet` 物体检测模型配置此 `nvinfer` 插件。此插件会将*物体分类*、*边界框坐标* 等推理结果附加到元数据中。
    * **注意**：推理行为需通过配置文件完成设置。
    <p><img src="images/nvinfer.png" width='720'></p>
* [fakesink](https://gstreamer.freedesktop.org/documentation/coreelements/fakesink.html) 插件可用作数据流终止传输的工作流水线的终点。
    * GStreamer 支持时钟同步机制，对于本地视频文件，建议将 `sync` 属性设置为 `1` 以流畅播放视频，对于 IP 摄像头等实时源，则将该属性设置为 `0`，因为这些实时源以静态速率提供帧。

*如需有关插件的更多信息，请参阅 [DeepStream 插件指南](https://docs.nvidia.com/metropolis/deepstream/dev-guide/index.html#plugins-development-guide)。*

此外，我们还将在 `nvinfer` 插件的源衬垫 (Pad) 上安装**回调函数**，以便在应用中访问元数据。应用程序随后可以使用此元数据解决特定问题（在本例中，即统计汽车、行人、路标和两轮车的数量）。

<a name='s3.2'></a>
### 初始化 GStreamer 和工作流水线 ###
首先，使用 `Gst.init()` 初始化 GStreamer，并将 `Gst.Pipeline` 对象实例化为 `pipeline`，以加入工作流图中显示的所有元件。

In [ ]:
# DO NOT CHANGE THIS CELL
# Import necessary GStreamer libraries and DeepStream python bindings
import gi
gi.require_version('Gst', '1.0')
from gi.repository import GObject, Gst, GLib
import pyds

# Initialize GStreamer
Gst.init(None)

# Create Pipeline element that will form a connection of other elements
pipeline=Gst.Pipeline()
print('Created pipeline')

<a name='s3.3'></a>
### 创建工作流水线元件 ###
使用 `Gst.ElementFactory.make(factoryname, name)`（请参阅[文档](https://lazka.github.io/pgi-docs/Gst-1.0/classes/ElementFactory.html#Gst.ElementFactory.make)）在工作流水线中创建每个元件。可以根据需要使用 `set_property(prop_id, value)`（请参阅[文档](https://gstreamer.freedesktop.org/documentation/plugin-development/basics/args.html?gi-language=c)）和所需参数配置元件。接下来，使用 `Gst.Pipeline.add(Gst.Element)` 将所有相关元件添加到工作流中。

In [ ]:
# DO NOT CHANGE THIS CELL
# Create Source element for reading from a file and set the location property
source=Gst.ElementFactory.make("filesrc", "file-source")
source.set_property('location', target_video_path)

# Create H264 Parser with h264parse as the input file is an elementary h264 stream
h264parser=Gst.ElementFactory.make("h264parse", "h264-parser")

# Create Decoder with nvv4l2decoder for accelerated decoding on GPU
decoder=Gst.ElementFactory.make("nvv4l2decoder", "nvv4l2-decoder")

# Create Streamux with nvstreammux to form batches for one or more sources and set properties
streammux=Gst.ElementFactory.make("nvstreammux", "stream-muxer")
streammux.set_property('width', 888) 
streammux.set_property('height', 696) 
streammux.set_property('batch-size', 1)

# Create Primary GStreamer Inference Element with nvinfer to run inference on the decoder's output after batching
pgie=Gst.ElementFactory.make("nvinfer", "primary-inference")

# Create Sink with fakesink as the end point of the pipeline
fakesink=Gst.ElementFactory.make('fakesink', 'fakesink')
fakesink.set_property('sync', 1)
print('Created elements')

<p><img src='images/important.png' width=720></p>

即使源视频的输出维度为 **882** x **692**，`nvstreammux` 插件也要将其设置为最接近该值的 **8** 的倍数，因此我们必须将其设置为 **888** x **696**。否则，此插件会自动设置该属性。系统会将额外的像素填充为黑色。将 `batch-size` 设置为每批 **1** 帧。_建议_将 `batch-size` 设置为等于输入源数量的值。

In [ ]:
# DO NOT CHANGE THIS CELL
# Add elements to pipeline
pipeline.add(source)
pipeline.add(h264parser)
pipeline.add(decoder)
pipeline.add(streammux)
pipeline.add(pgie)
pipeline.add(fakesink)
print('Added elements to pipeline')

<a name='e1'></a>
#### 练习 #1 - 了解 Gst-nvinfer 配置文件 ####
`nvinfer` 插件需要通过[配置文件](./spec_files/pgie_config_trafficcamnet_03.txt)进行配置，我们已事先完成此配置工作。请查看其结构展开深入了解。

以下属性很重要，有助于了解详情：
* 从 NGC 下载或使用 TAO 工具包训练的模型的相关属性：
    * `tlt-encoded-model` - TAO 工具包编码模型的路径名。
    * `tlt-model-key` - TAO 工具包编码模型的模型加载密钥。
    * `labelfile-path` - 包含模型标签的文本文件的路径名。这些标签必须以换行符分隔，才能用于物体检测模型。
    * `uff-input-blob-name` - UFF 文件中输入 Blob 的名称。
    * `output-blob-names` - 输出层名称数组。
    * `input-dims` - 模型输入的维度，即[通道、高度、宽度、输入顺序]，其中 NCHW 的输入顺序为 0。
    * `net-scale-factor` - 像素归一化因子 *(默认值为 1)* 。

* 推荐属性：
    * `batch-size` - 一同接受批量推理操作的帧数 *(默认值为 1)* 。

* 检测器的必填属性：
    * `num-detected-classes` - 网络检测到的类数量。

* 检测器的可选属性：
    * `cluster-mode` - 使用的聚类算法 *(默认值为 0，即矩形分组)* 。如需更多信息，请参阅文档中 [nvinfer 支持的聚类算法](https://docs.nvidia.com/metropolis/deepstream/dev-guide/text/DS_plugin_gst-nvinfer.html#cluster-mode-info)的相关部分。
    * `nms-iou-thresholds`：两次建议之间的最高 IOU 评分，之后将拒绝置信度较低的建议。
    * `pre-cluster-thresholds`：要在开展聚类计算之前应用的检测阈值。
    * `topk`：只保留检测分数最高的前 K 个物体。

* 其他可选属性：
    * `network-mode` - 用于推理的数据格式 *（0 表示 FP32，1 表示 INT8，2 表示 FP16 模式，| 默认值为 0，即 FP32）* 。
    * `process-mode` - 插件的运行模式 *（主要或次要)* *（默认值为 1，即主要模式）* 。
    * `model-color-format` - 模型所需的色彩格式 *（默认值为 0，即 RGB）* 。
    * `interval` - 跳过推理的连续批数 *（仅针对主要模式，| 默认值为 0）* 。
    * `gie-unique-id` - 要分配给 GIE 的唯一 ID，用于帮助应用和其他元件识别检测到的边界框和标签 *（默认值为 0）* 。
    * `gpu-id` - 执行预处理/推理所用 GPU 的设备 ID *（仅限 dGPU）* 。

您可以在[模型卡](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/tao/models/trafficcamnet)上找到所需的大部分信息：

<p><img src='images/encryption_key.png' width=480></p>
<p><img src='images/model_card.png' width=720></p>
<p><img src='images/blob_names.png' width=360></p>

In [ ]:
# DO NOT CHANGE THIS CELL
!cat spec_files/pgie_config_trafficcamnet_03.txt

**注意**：配置文件中的值会被通过 `GObject` 属性设置的值覆盖。务必谨记的另一点是：推荐的属性为主要检测器的专属属性。您可能需要为辅助设备或分类器配置其他属性。

创建配置文件后，可以使用 `set_property()` 设置 `nvinfer` 插件的配置文件路径。

In [ ]:
# DO NOT CHANGE THIS CELL
# Set the configuration-file-path property for nvinfer
pgie.set_property('config-file-path', '/dli/task/spec_files/pgie_config_trafficcamnet_03.txt')

<a name='s3.4'></a>
### 关联工作流水线元件 ###
最后，按照数据在工作流水线中的流式传输的顺序，使用 `Gst.Element.link(Gst.Element)` 关联所有相关元件。
<p><img src='images/important.png' width=720></p>

将特定源连接至 `nvstreammux`（复用器）时，必须手动关联通过 `get_static_pad('src')` 获取的输入源衬垫 (Pad) 与复用器使用 `get_request_pad()` 和衬垫 (Pad) 模板 `sink_%u` 提供的新请求衬垫 (Pad)。这样才能从多个源中向 `nvstreammux` 插件输入数据。

In [ ]:
# DO NOT CHANGE THIS CELL
# Link elements in the pipeline
source.link(h264parser)
h264parser.link(decoder)

# Link decoder source pad to streammux sink pad
decoder_srcpad=decoder.get_static_pad("src")    
streammux_sinkpad=streammux.get_request_pad("sink_0")
decoder_srcpad.link(streammux_sinkpad)

# Link the rest of the elements in the pipeline
streammux.link(pgie)
pgie.link(fakesink)
print('Linked elements in pipeline')

<a name='s3.5'></a>
### 探查 ###
如前文所述，我们使用探针访问元数据，而探针就是与元件衬垫 (Pad) 进行交互的回调函数。若要访问元数据，可以使用 DeepStream Python 绑定 `pyds`，例如 `pyds.gst_buffer_get_nvds_batch_meta()`。DeepStream 使用可扩展的标准元数据结构。基本元数据结构 `NvDsBatchMeta` 以 `Gst-nvstreammux` 插件中创建的批量级元数据开头（参见下文）。我们查找的元数据可以通过 `NvDsBatchMeta` > `NvDsFrameMeta` > `NvDsObjectMeta` 进行访问。系统会以 `GList` 形式存储某些元数据实例，而这需要使用 `pyds.NvDsFrameMeta.cast()` 或 `pyds.NvDsObjectMeta.cast()` 将数据投射到相应的结构中。`NvDsObjectMeta` 中包含深度学习神经网络的推理结果，具体视配置而定。
<p><img src='images/DS_plugin_metadata.png' width=480></p>

系统以编程的方式将探针函数的返回值处理为 `Gst.PadProbeReturn.OK`，这表示探针返回值“正常”，并且探针已安装到位。对于更复杂的情况，还可以考虑使用其他[返回值](https://lazka.github.io/pgi-docs/Gst-1.0/enums.html#Gst.PadProbeReturn)选项。

在用户定义的回调函数 `pgie_source_pad_buffer_probe` 中，我们可以遍历附加至缓冲区的所有元数据类型。DeepStream 插件会将 `NVDS_META_FRAME_INFO` 类型的元数据附加至缓冲区。我们可访问物体元数据，以统计帧中的物体数量，并输出边界框坐标。在本例中，我们使用 4 类检测器（车辆、行人、两轮车和路标）。

In [ ]:
# DO NOT CHANGE THIS CELL
# Declare list to hold count data
obj_counts=[]

# Define the Probe Function
def pgie_src_pad_buffer_probe(pad, info):
    gst_buffer=info.get_buffer()

    # Retrieve batch metadata from the gst_buffer
    # Note that pyds.gst_buffer_get_nvds_batch_meta() expects the
    # C address of gst_buffer as input, which is obtained with hash(gst_buffer)
    batch_meta=pyds.gst_buffer_get_nvds_batch_meta(hash(gst_buffer))
    l_frame=batch_meta.frame_meta_list
    
    # Iterate through each frame in the batch metadata until the end
    while l_frame is not None:
        try:
            frame_meta=pyds.NvDsFrameMeta.cast(l_frame.data)
        except StopIteration:
            break

        frame_num=frame_meta.frame_num
        num_obj=frame_meta.num_obj_meta
        l_obj=frame_meta.obj_meta_list
        
        print("Frame Number={} Number of Objects={}".format(frame_num, num_obj))
        
        # Append number of objects a list 
        obj_counts.append(num_obj)
        
        # Iterate through each object in the frame metadata until the end
        while l_obj is not None:
            try:
                obj_meta=pyds.NvDsObjectMeta.cast(l_obj.data)
                print('\t Object: {} - Top: {}, Left: {}, Width: {}, Height: {}'.format(obj_meta.obj_label, \
                                                                                        round(obj_meta.rect_params.top), \
                                                                                        round(obj_meta.rect_params.left), \
                                                                                        round(obj_meta.rect_params.width), \
                                                                                        round(obj_meta.rect_params.height)))
            except StopIteration:
                break
            
            try: 
                l_obj=l_obj.next
            except StopIteration:
                break
        
        try:
            l_frame=l_frame.next
        except StopIteration:
            break
    return Gst.PadProbeReturn.OK

在定义工作流并连接元件后，可使用 `Gst.Pad.add_probe()` 在 `nvinfer` 插件的源衬垫 (Pad) 上安装回调函数。附加后，探针会在衬垫 (Pad) 上发生数据传递时发出通知。创建探针后，可将 `GST_PAD_PROBE_TYPE_BUFFER` 或 `GST_PAD_PROBE_TYPE_BUFFER_LIST` 传递至 `gst_pad_add_probe()`。设计回调函数旨在与单个缓冲区搭配使用，因此要使用 `Gst.PadProbeType.BUFFER`。

In [ ]:
# DO NOT CHANGE THIS CELL
# Add probe to inference plugin's source
pgie_src_pad=pgie.get_static_pad('src')
probe_id=pgie_src_pad.add_probe(Gst.PadProbeType.BUFFER, pgie_src_pad_buffer_probe)
print('Attached probe')

<a name='s3.6'></a>
### 启动工作流水线 ###
工作流水线中的总线可用于侦听消息。运行 GLib/Gtk+ `MainLoop`（或定期迭代默认的 GLib 主上下文），并使用 `add_signal_watch()` 将侦听/消息处理器附加至总线。这样，`GLib.Mainloop` 就会检查总线是否有新消息，并随时通知我们。我们也可以将回调函数定义为 `bus_call` 来实现消息处理器。每当工作流水线向总线发送消息时，系统就会调用此处理器。此回调函数的返回值应为 `True`，这样才能将消息附加至总线。由于检查总线消息的回调函数可用于许多 DeepStream 应用，我们可将其重构为 [Python 脚本](common/bus_call.py) (`common/bus_call.py`)，以便使用。当使用 `inpsect.getsource()` 查看函数定义时，我们发现此简单函数可检查消息类型并对其展开相应处理。然后，将工作流水线设置为 `PLAYING` 状态，并运行 `MainLoop`。最后，在创建完工作流水线时，将其设置为 `NULL` 状态，以执行清除操作。

In [ ]:
# DO NOT CHANGE THIS CELL
from common.bus_call import bus_call

# Inspect the definition for bus_call
from inspect import getsource
print(getsource(bus_call))

In [ ]:
# DO NOT CHANGE THIS CELL
# Create an event loop
loop=GLib.MainLoop()

# Feed GStreamer bus messages to loop
bus=pipeline.get_bus()
bus.add_signal_watch()
bus.connect("message", bus_call, loop)
print('Added bus message handler')

In [ ]:
# DO NOT CHANGE THIS CELL
# Start play back and listen to events
print("Starting pipeline")
pipeline.set_state(Gst.State.PLAYING)
try:
    loop.run()
except:
    pass

# Cleaning up as the pipeline comes to an end
pipeline.set_state(Gst.State.NULL)

<a name='s3.7'></a>
### 分析推理结果 ###
在 DeepStream 工作流水线完成视频处理后，我们可以汇总推理结果。

In [ ]:
# DO NOT CHANGE THIS CELL
import pandas as pd

# Export data to a Pandas Series
obj_count_df=pd.Series(obj_counts, name='Object Count')
obj_count_df.index.name='Frame Number'

# Plot the Series
obj_count_df.plot(
    linestyle='none', 
    marker='.', 
    figsize=(15, 5), 
    ylim=[-.1, 3.1], 
    title='Object Count Over Time'
)

<p><img src='images/check.png' width=720></p>
恭喜！您已完成第一个 DeepStream 工作流水线的构建。在此流水线中添加更多功能之前，先看看如何评估其性能。

<a name='s4'></a>
## 性能分析 ##
在本节中，我们将查看用于评估 DeepStream 流水线性能的各种指标：
* 吞吐量
* 延迟
* 硬件利用率

到目前为止，我们可以将代码重构为 [Python 脚本](sample_apps/app_3_pt_1.py) (`app_3_pt_1.py`)，以便执行。如需更多信息，请参阅文档字符串。此脚本还支持以模组形式导入，以便使用 `building_simple_pipeline()` 构建简单的 DeepStream 工作流水线。如需更多信息，请参阅文档字符串。

In [ ]:
# DO NOT CHANGE THIS CELL
# Import app_3_pt_1.py from sample_apps directory
import sample_apps.app_3_pt_1

# Print the docstring 
print(sample_apps.app_3_pt_1.__doc__)

In [ ]:
# DO NOT CHANGE THIS CELL
from sample_apps.app_3_pt_1 import build_simple_pipeline

# Print the docstring
print(build_simple_pipeline.__doc__)

In [ ]:
# DO NOT CHANGE THIS CELL
# Test run the Python script
!python sample_apps/app_3_pt_1.py data/sample_30.h264

<a name='s4.1'></a>
### 使用探针探测吞吐量 ###
**吞吐量**是指特定时间内处理的数据量。此指标可被视为执行流处理时成功传输数据的平均速率。DeepStream 应用的吞吐量应符合输入视频流的要求。我们可以使用探针估量吞吐量，了解数据流的情况，并将其表示为每秒帧数。下面我们将修改探针函数，以探测通过 `nvinfer` 插件传递帧所需要的时间。利用时间的[倒数](https://en.wikipedia.org/wiki/Reciprocal)，可以获得帧速率近似值。尽管我们会输出单个视频流的帧率，但我们可以对此操作进行扩展，生成一些指标，例如每个视频流在某个粒度级别的最小、最大和平均帧吞吐量。

In [ ]:
# DO NOT CHANGE THIS CELL
import time

frame_rates=[]

# Define the Probe Function
def pgie_src_pad_buffer_probe_fps(pad, info):
    global start
    frame_number=0
    gst_buffer=info.get_buffer()
    
    batch_meta=pyds.gst_buffer_get_nvds_batch_meta(hash(gst_buffer))
    l_frame=batch_meta.frame_meta_list
    
    # Iterate through each frame in the batch metadata until the end
    while l_frame is not None:
        now=time.time()
        try:
            frame_meta=pyds.NvDsFrameMeta.cast(l_frame.data)
        except StopIteration:
            break
        frame_number=frame_meta.frame_num
        
        # Take the reciprocal of the time difference as frame rate
        frame_rate=round(1/(now-start), 2)
        print('FPS: {} @ Frame {}.'.format(frame_rate, frame_number))
        
        # Add frame rate to frame_rates list
        frame_rates.append(frame_rate)
        
        try:
            l_frame=l_frame.next
        except StopIteration:
            break
        start=now
    return Gst.PadProbeReturn.OK

我们将使用 [app_3_pt_1.py](sample_apps/app_3_pt_1.py) 脚本中的 `build_simple_pipeline()` 构建简单的工作流水线，并将更新的探针插入主要推理插件的源衬垫 (Pad)。

In [ ]:
# DO NOT CHANGE THIS CELL
# Standard GStreamer initialization
Gst.init(None)

# Build pipeline
pipeline=build_simple_pipeline('data/sample_30.h264')
print('Successfully created a {} object'.format(type(pipeline)))

# Get the nvinfer plugin by name
pgie=pipeline.get_by_name('primary-inference')

# Add probe to inference plugin's source
pgie_src_pad=pgie.get_static_pad('src')
pgie_src_pad.add_probe(Gst.PadProbeType.BUFFER, pgie_src_pad_buffer_probe_fps)
print('Attached Probe')

In [ ]:
# DO NOT CHANGE THIS CELL
# Create MainLoop and add a signal watch
loop=GLib.MainLoop()
bus=pipeline.get_bus()
bus.add_signal_watch()
bus.connect("message", bus_call, loop)
print('Added bus message handler')

与前述操作一样，我们可以使用 `GLib.MainLoop` 开始播放。

In [ ]:
# DO NOT CHANGE THIS CELL
# Start Pipeline
print("Starting pipeline")
pipeline.set_state(Gst.State.PLAYING)
start=time.time()
try:
    loop.run()
except:
    pass

# Cleaning up as the pipeline comes to an end
pipeline.set_state(Gst.State.NULL)

In [ ]:
# DO NOT CHANGE THIS CELL
# Export data to a Pandas Series
frame_rates_df=pd.Series(frame_rates, name='Frame Rate')
frame_rates_df.index.name='Frame Number'

# Plot the Series
frame_rates_df.plot(
    linestyle='none', 
    marker='.', 
    figsize=(15, 5), 
    ylim=[-.1, 50], 
    title='Frame Rate Over Time')

<p><img src='images/check.png' width=720></p>
根据分析，不难看出我们以符合所需输入视频流要求的高帧速率 30 FPS（帧/秒）运行。鉴于吞吐量不同，强烈建议您分析应用的帧速率最小值、最大值和平均值，以便了解任何可能发生的非预期性能问题。

<a name='s4.2'></a>
### 使用调试日志测量延迟 ###
**延迟**是指流从工作流水线中的一个元件进入另一个元件之间的时间差。通常情况下，低延迟自然是理想状态，这对时间要求严苛的实时工作流水线格外重要。在视频分析的语境下，延迟是指处理输入流所需的时间。我们可以使用 GStreamer 调试功能测量 DeepStream 工作流中的延迟。调试日志很有用，但也很冗长。我们可以使用下表（参见 [GStreamer 文档](https://gstreamer.freedesktop.org/documentation/tutorials/basic/debugging-tools.html)）指定所需输出量。将 `GST-DEBUG` 环境变量设置为 `GST_SCHEDULING:7` 后，在运行 DeepStream 工作流时会收到 `trace` 日志，其中包含有关何时修改缓冲区的详细信息。我们可以使用这些时间戳推导工作流中不同元件之间的延迟。

| # | 名称    | 说明                                                    |
|-|---------|----------------------------------------------------------------|
| 0 | none    | 未输出调试信息。|
| 1 | ERROR   | 记录所有致命错误。这些错误不允许  |
|   |         | 核心或元件执行请求的操作。如果          |
|   |         | 应用经过编程可处理触发错误的情况，则      |
|   |         | 仍然可以恢复。|
| 2 | WARNING | 记录所有警告。通常是指一些用户会遇到的预期问题，|
|   |         | 并不致命。|
| 3 | FIXME   | 记录所有 "fixme" 消息。通常是指触发了已知|
|   |         | 不完整的代码路径。在大多数情况下，应用   |
|   |         | 可以运行，但可能会造成特定实例问题。|
| 4 | INFO    | 记录所有信息消息。这些消息通常用于记录  |
|   |         | 系统中仅发生一次的事件，或影响很大   |
|   |         | 但很少在此级别进行记录的事件。|
| 5 | DEBUG   | 记录所有调试消息。这些消息是一般调试消息，用于记录  |
|   |         | 只是在对象的生命周期内发生次数有限的    |
|   |         | 事件；其中包括设置、拆解、更改    |
|   |         | 参数等事件。|
| 6 | LOG     | 记录所有日志消息。这些消息用于记录      |
|   |         | 在对象生命周期内反复发生的事件；其中包括   |
|   |         | 流式传输和稳定状态情况。可用于记录    |
|   |         | 特定元件各缓冲区的日志消息。|
| 7 | TRACE   | 记录所有追踪消息。这些消息的生成频率    |
|   |         | 很高。例如，每次修改     |
|   |         | GstMiniObject（如 GstBuffer 或 GstEvent）的引用计数时，都会生成此  |
|   |         | 消息。|
| 9 | MEMDUMP | 记录所有内存转储消息。这是较重要的日志记录，其中|
|   |         | 可能包括内存块内容的转储。|

使用 GStreamer 时，可自定义调试信息处理器，但如果使用默认的调试信息处理器，则调试输出中的各行内容如下：

截取信息的格式如下：

| 示例          | 说明                                                 |
|-|-----------------------------------------------------------|
|0:00:00.868050000 | 程序启动时的时间戳，采用 HH:MM:SS.sssssssss|
|                  | 格式。|
|1592              | 发出消息所使用的进程 ID。当问题 |
|                  | 涉及多个进程时很有用。|
|09F62420          | 发出消息所使用的线程 ID。当问题  |
|                  | 涉及多个线程时很有用。|
|WARN              | 消息的调试级别。|
|filesrc           | 消息的调试类别。|
|gstfilesrc.c:1044 | GStreamer 源代码中发出此消息的   |
|                  | 源文件和行。|
|gst_file_src_start| 发出消息的函数。|
|<filesrc0>        | 发出消息的对象的名称。此对象可以是  |
|                  | 元件、衬垫 (Pad) 或其他对象。在使用   |
|                  | 多个同类元件且需要加以区分时，此设置|
|                  | 很有用。使用名称属性命名元件   |
|                  | 可增强调试输出的可读性，但 GStreamer       |
|                  | 默认为每个新元件分配一个唯一名称。|
| error: No such   |                                                           |
| file ....| 实际的消息内容。|

运行以下单元可重新运行 [app_3_pt_1.py](sample_apps/app_3_pt_1.py) 并生成日志。我们可以预览输出 [trace.log](logs/trace.log) 文件的前几行。为使日志文件发挥作用，我们必须解析该文本。为便于演示，我们已加入简单脚本来完成此任务。

In [ ]:
# DO NOT CHANGE THIS CELL
!GST_DEBUG='GST_SCHEDULING:7' GST_DEBUG_FILE='/dli/task/logs/trace.log' python sample_apps/app_3_pt_1.py data/sample_30.h264

[trace.log](./logs/trace.log) 文件很大，我们仅在此展示了小部分文件，这部分突出显示了缓冲区进入解码器插件的时间，以及缓冲区进入下一个输入的时间。

In [ ]:
# DO NOT CHANGE THIS CELL
!head /dli/task/logs/trace.log

In [ ]:
# DO NOT CHANGE THIS CELL
# Import dependencies
import re

trace_log=[]
# Column headers per documentation
headers=['time_stamp', 'process_id', 'thread_id', 'level', 'category', 'src_file_line', 'function', 'object_name', 'message']

# Helper function to remove ANSI escape sequences
def escape_ansi(line):
    ansi_escape = re.compile(r'(?:\x1B[@-_]|[\x80-\x9F])[0-?]*[ -/]*[@-~]')
    return ansi_escape.sub('', line)

# Open trace.log
with open('/dli/task/logs/trace.log') as f: 
    # Read file
    lines=f.readlines()
    # Iterate through each line
    for each_line in lines: 
        # Exclude the last character, which is a newline (\n) character
        current_line=escape_ansi(each_line[:-1])
        # Split based on white space(s), keeping in mind that src_file, line, function, and object are concatenated together
        time_stamp, process_id, thread_id, level, category, src_file_line_function_object, message=re.split(' +', current_line, maxsplit=6)
        # Split src_file, line, function, and object based on the semicolon character
        src_file, line, function, object=src_file_line_function_object.split(':', maxsplit=3)
        # Add all data to the trace_log list
        trace_log.append([time_stamp, process_id, thread_id, level, category, f'{src_file}:{line}', function, object, message])

# Export data to a DataFrame
df=pd.DataFrame(trace_log, columns=headers)
# Preview the dataframe
df.head()

每行都截取了消息的时间戳以及消息本身所对应的帧的时间戳。以下脚本解析了正在处理的帧的时间戳。通过检查与同一帧相关的消息的时间戳，我们可以分析数据流从工作流中的一个元件进入另一个元件之间的时间差。

In [ ]:
# DO NOT CHANGE THIS CELL
# Iterate through rows backwards to get the time stamp
for idx, row in df[::-1].iterrows(): 
    # Time stamp is pts if object is a sink
    if row['object_name'] in ['<stream-muxer:sink_0>', '<primary-inference:sink>', '<fakesink:sink>']: 
        try: 
            df.loc[idx, 'frame_ts']=re.findall('pts \d+:\d+:\d+.\d+', row['message'])[0].split('pts ')[-1]
        except: 
            pass
    # Time stamp is dts if object is a decoder sink
    elif row['object_name']=='<nvv4l2-decoder:sink>': 
        try: 
            ts=re.findall('dts \d+:\d+:\d+.\d+', row['message'])[0].split('dts ')[-1]
            if ts: 
                df.loc[idx, 'frame_ts']=ts
                decoder_offset=re.findall('offset \d+', row['message'])[0].split('offset ')[-1]
        except: 
            pass
    # Time stamp is same as dts of decoder with same offset for file source
    elif row['object_name']=='<file-source:src>':
        try: 
            src_offset=re.findall('offset \d+', row['message'])[0].split('offset ')[-1]
            if src_offset==decoder_offset: 
                df.loc[idx, 'frame_ts']=ts
        except: 
            pass

In [ ]:
# DO NOT CHANGE THIS CELL
time_df=df[['time_stamp', 'object_name', 'frame_ts']].dropna().drop_duplicates(subset=['object_name', 'frame_ts'])

# Pivot dataframe
time_df=time_df.pivot(index='object_name', values='time_stamp', columns='frame_ts')

# Leave time stamp as only seconds 
time_df.columns=[float(each_column.split(':')[2]) for each_column in time_df.columns]

# Display time_df
time_df=time_df.sort_values(0.0).applymap(lambda x: float(x.rsplit(':')[2]))
print('Time Stamp when Buffer Arrives (seconds)')
display(time_df) 

In [ ]:
# DO NOT CHANGE THIS CELL
# Calculate time difference as processed time
diff_df=-time_df.diff(-1).T

# Plot results
diff_df.iloc[:, :-1].plot(figsize=(15, 5)).legend(loc='upper right')

<p><img src='images/check.png' width=720></p>
通过查看视频流从流水线中的一个元件进入另一个元件之间的时间差，即可计算出延迟。使用追踪日志，可以确定流水线中的插件是否运行缓慢。

<a name='s4.3'></a>
### 使用 NVIDIA 系统管理接口查看 GPU 利用率 ###
使用 `nvidia-smi` 可以查看应用运行时的 **GPU 利用率**指标。运行以下单元可重新运行 [app_3_pt_1.py](sample_apps/app_3_pt_1.py)，同时记录 `nvidia-smi` 的结果。我们将为 `nvidia-smi` 使用以下选项：
* `dmon -i 0`：参照以逗号分隔的设备列表，报告所选择设备的默认指标（设备监测）。由于索引为 0 的 GPU 是我们使用的唯一 GPU，因此我们将报告该 GPU 的默认指标。
* `-s ucmt`：选择要显示的指标。在本例中，我们使用 ucmt 来标示所需指标。
    * u：利用率（SM、显存、编码器和解码器利用率，以 % 表示）
    * c：Proc 和 Mem 时钟（以 MHz 为单位）
    * m：帧缓冲区和 Bar1 显存使用量（以 MB 为单位）
    * t：以 MB/s 为单位的 PCIe Rx 和 Tx 吞吐量（针对 Maxwell 及更优架构）
* `-c 10`：配置监控迭代。系统将显示指定样本数量的数据并退出迭代循环。在本例中，我们选择 10 次迭代。

In [ ]:
# DO NOT CHANGE THIS CELL
!nvidia-smi dmon -i 0 \
                 -s ucmt \
                 -c 100 > '/dli/task/logs/smi.log' \
&python sample_apps/app_3_pt_1.py data/sample_30.h264

可以在终端查看 `smi.log`，也可以[单独](./logs/smi.log)查看。

In [ ]:
# DO NOT CHANGE THIS CELL
!cat /dli/task/logs/smi.log

<p><img src='images/check.png' width=720></p>

利用率指标能够体现每个 GPU 在一段时间内的使用情况，可用于确定应用在系统中的 GPU 利用率。具体而言，`sm` 列追踪在过去的采样周期内一个或多个内核使用 GPU 的时间百分比。fb 报告 GPU 的帧缓冲区显存使用情况。如需了解详情，请点击[此处](http://developer.download.nvidia.com/compute/DCGM/docs/nvidia-smi-367.38.pdf)以获取更全面的文档。

<a name='s5'></a>
## 构建视频 AI 应用 - 第二部分 ##
在第二部分，我们将在 DeepStream 工作流水线中添加更多功能，以创建复合视频并将输出写入文件。

<a name='s5.1'></a>
### 工作流水线组件 ###
这是应用的工作流水线架构。我们将使用物体检测网络来识别和定位各帧中的汽车。我们来了解下各个插件：
<p><img src="images/deepstream_pipeline.png" width='720'></p>

* 首先，`fileSrc` 组件可读取压缩文件中的数据，并将其传入解码器。此外，还有其他可用的数据读取插件，这些插件可读取摄像头、RTSP 流和磁盘等各类源中的数据。
* `h264parse` 适用于输入文件是基本 h264 流的情况
* `nvv4l2decoder` 可解码传入数据，并将输出帧传递至 `nvinfer` 组件。
* `nvstreammux` 组件是必要的流复用器。此视频复用器有助于接收 n 个输入流，并将其转换为连续的批量帧。此组件在此过程中使用低级别 API 访问 GPU 和 CPU。使用 `width`、`height` 和 `batch-size` 作为参数。
* `nvinfer` 组件可使用深度学习物体检测模型进行配置。完成推理后，此组件会将输出元数据（*物体类型、边界框等*）附加至缓冲区。
    * **注意：推理行为需通过配置文件完成设置**
* `nvvideoconvert` 根据 nvosd 的要求将视频格式从 NV12 转换为 RGBA。此组件可以将格式从 YUV 转换为 RGBA/BRGA，扩大分辨率和执行图像旋转操作。
* 然后，系统会将缓冲区中的视频数据及其元数据发送至屏幕显示 (`nvdsosd`) 组件，以绘制物体的边界框并提供属性描述文本。
* 另一个 `nvvideoconvert` 可将格式转换为正确的格式。
* `capsfilter` 可强制执行数据格式
* `encoder` 可将帧转换为视频格式
* `mpeg4videoparse` 可将 MPEG-4 解析为基本视频流
* `qtmux` 可将流合并为 QuickTime(.mov) 文件。
* `filesink` 可将传入数据写入本地文件系统的文件中。顾名思义，汇点就是正常流终止传输的工作流终点。

*如需有关插件的更多信息，请参阅 [DeepStream 插件指南](https://docs.nvidia.com/metropolis/deepstream/dev-guide/index.html#plugins-development-guide)。*

此外，我们还将在 `nvdsosd` 组件的接收衬垫 (Pad) 上安装**回调函数**，以便在应用中访问元数据。应用随后可以使用此元数据解决特定问题（在本例中，即统计汽车的数量）。

<a name='s5.2'></a>
### 初始化 GStreamer ###
接下来，使用 `Gst.init()` 初始化 GStreamer，并将 `Gst.Pipeline` 对象实例化为 `pipeline`，以在其中加入工作流图中显示的所有元件。

In [ ]:
# DO NOT CHANGE THIS CELL
# Initialize GStreamer
Gst.init(None)

# Build pipeline
pipeline=build_simple_pipeline('data/sample_30.h264')
print('Successfully created a {} object'.format(type(pipeline)))

<a name='s5.3'></a>
### 创建和连接工作流水线元件 ###
然后，使用 `Gst.ElementFactory.make(factoryname, name)` 创建流水线中所需的所有元件（请参阅[文档](https://lazka.github.io/pgi-docs/Gst-1.0/classes/ElementFactory.html#Gst.ElementFactory.make)）。使用 `set_property(prop_id, value)` 配置元件及所需参数（请参阅[文档](https://gstreamer.freedesktop.org/documentation/plugin-development/basics/args.html?gi-language=c)），以便每个组件执行一项必要任务。接下来，使用 `Gst.Pipeline.add` 将所有相关元件添加到工作流水线中。

In [ ]:
# DO NOT CHANGE THIS CELL
# Remove Fakesink
fakesink=pipeline.get_by_name('fakesink')
pipeline.remove(fakesink)

# Create Convertor to convert from YUV to RGBA as required by nvdsosd
nvvidconv1=Gst.ElementFactory.make("nvvideoconvert", "convertor1")

# Create OSD with nvdsosd to draw on the converted RGBA buffer
nvosd=Gst.ElementFactory.make("nvdsosd", "onscreendisplay")

# Create Convertor to convert from RGBA to I420 as required by encoder
nvvidconv2=Gst.ElementFactory.make("nvvideoconvert", "convertor2")

# Create Capsfilter to enforce frame image format
capsfilter=Gst.ElementFactory.make("capsfilter", "capsfilter")
caps=Gst.Caps.from_string("video/x-raw, format=I420")
capsfilter.set_property("caps", caps)

# Create Encoder to encode I420 formatted frames using the MPEG4 codec
encoder = Gst.ElementFactory.make("avenc_mpeg4", "encoder")
encoder.set_property("bitrate", 2000000)

# Create Sink and set the location for the output file
filesink=Gst.ElementFactory.make('filesink', 'filesink')
filesink.set_property('location', 'output_03_encoded.mpeg4')
filesink.set_property("sync", 1)
print('Created elements')

In [ ]:
# DO NOT CHANGE THIS CELL
# Add elements to pipeline
pipeline.add(nvvidconv1)
pipeline.add(nvosd)
pipeline.add(nvvidconv2)
pipeline.add(capsfilter)
pipeline.add(encoder)
pipeline.add(filesink)
print('Added elements to pipeline')

<a name='s5.4'></a>
### 关联工作流水线元件 ###
最后，按照数据在工作流水线中的流式传输的顺序，使用 `Gst.Element.link(Gst.Element)` 关联所有相关元件。

In [ ]:
# DO NOT CHANGE THIS CELL
# Get the nvinfer plugin by name
pgie=pipeline.get_by_name('primary-inference')

# Link elements together
pgie.link(nvvidconv1)
nvvidconv1.link(nvosd)
nvosd.link(nvvidconv2)
nvvidconv2.link(capsfilter)
capsfilter.link(encoder)
encoder.link(filesink)
print('Linked elements in pipeline')

<a name='s5.5'></a>
### 探查 ###
如前文所述，我们使用探针访问元数据，而探针就是与元件衬垫 (Pad) 进行交互的回调函数。若要访问元数据，可以使用 DeepStream Python 绑定 `pyds`，例如 `pyds.gst_buffer_get_nvds_batch_meta()`。我们查找的元数据位于 `NvDsBatchMeta` > `NvDsFrameMeta` > `NvDsObjectMeta` 下面。系统会以 `GList` 形式存储某些元数据实例，而这需要使用 `pyds.NvDsFrameMeta.cast()` 和 `pyds.NvDsObjectMeta.cast(()` 将数据投射到相应的结构中。`NvDsObjectMeta` 中包含深度学习神经网络的推理结果，具体视配置而定。

`Gst-nvdsosd` 插件可绘制边界框、文本和兴趣区域 (RoI) 多边形，而上述内容均可通过元数据加以配置。也就是说，尚未在插件的接收衬垫 (Pad) 上配置显示元数据 (`NvDsDisplayMeta`)。我们使用的探针将帮助我们在帧元数据 (`NvDsFrameMeta`) 中添加此元数据。可以使用 `pyds.nvds_acquire_display_meta_from_pool()` 获取批量元数据的显示元数据。之后，可以修改/配置显示元数据，然后再使用 `nvds_add_display_meta_to_frame()` 将其添加到帧元数据中。

系统以编程的方式将探针函数的返回值处理为 `Gst.PadProbeReturn.OK`，这表示探针返回值“正常”，并且探针已安装到位。对于更复杂的情况，还可以考虑使用其他[返回值](https://lazka.github.io/pgi-docs/Gst-1.0/enums.html#Gst.PadProbeReturn)选项。

在回调函数 **osd_sink_pad_buffer_probe** 中，我们会遍历附加至缓冲区的所有元数据类型。DeepStream 组件会将类型元数据 `NVDS_META_FRAME_INFO` 附加至缓冲区。我们访问目标元数据以统计帧中的车辆数。在本例中，我们使用 4 类检测器（车辆、行人、两轮车和路标）。“车辆”类的 class_id 是 0。因此，我们使用 class_id = 0 统计物体数量。我们可以依照此法找出帧中的行人数。

In [ ]:
# DO NOT CHANGE THIS CELL
from random import random

# Define the Probe Function
def osd_sink_pad_buffer_probe(pad, info):
    gst_buffer=info.get_buffer()

    # Retrieve batch metadata from the gst_buffer
    # Note that pyds.gst_buffer_get_nvds_batch_meta() expects the
    # C address of gst_buffer as input, which is obtained with hash(gst_buffer)
    batch_meta=pyds.gst_buffer_get_nvds_batch_meta(hash(gst_buffer))
    l_frame=batch_meta.frame_meta_list

    # Iterate through each frame in the batch metadata until the end
    while l_frame is not None:
        try:
            frame_meta=pyds.NvDsFrameMeta.cast(l_frame.data)
        except StopIteration:
            break

        frame_number=frame_meta.frame_num
        num_rects=frame_meta.num_obj_meta
        l_obj=frame_meta.obj_meta_list
        
        # Iterate through each object in the frame metadata until the end
        while l_obj is not None:
            try:
                obj_meta=pyds.NvDsObjectMeta.cast(l_obj.data)
            except StopIteration:
                break
            
            # Set border color (red, green, blue, alpha) to random values
            obj_meta.rect_params.border_color.set(random(), random(), random(), random())
            
            try: 
                l_obj=l_obj.next
            except StopIteration:
                break

        # Acquire display metadata from pool and set number of labels to 1
        display_meta=pyds.nvds_acquire_display_meta_from_pool(batch_meta)
        display_meta.num_labels=1
        
        # Set text_params of the display metadata to local variable
        py_nvosd_text_params=display_meta.text_params[0]

        # Setting display text to be shown on screen
        py_nvosd_text_params.display_text="Frame Number={} Number of Objects={}".format(frame_number, num_rects)

        # Use pyds.get_string() to get display_text as string
        # Reading the display_text field here will return the C address of the
        # allocated string. Use pyds.get_string() to get the string content.
        print(pyds.get_string(py_nvosd_text_params.display_text))
        
        # Set the offsets where the string should appear
        py_nvosd_text_params.x_offset=10
        py_nvosd_text_params.y_offset=10

        # Set font, font-color (red, green, blue, alpha), and font-size
        py_nvosd_text_params.font_params.font_name="Serif"
        py_nvosd_text_params.font_params.font_size=15
        py_nvosd_text_params.font_params.font_color.set(1.0, 1.0, 1.0, 1.0)

        # Set text background color (red, green, blue, alpha)
        py_nvosd_text_params.set_bg_clr=1
        py_nvosd_text_params.text_bg_clr.set(0.0, 0.0, 0.0, 1.0)

        # Add to frame metadata
        pyds.nvds_add_display_meta_to_frame(frame_meta, display_meta)

        try:
            l_frame=l_frame.next
        except StopIteration:
            break
    return Gst.PadProbeReturn.OK

在定义工作流并关联元件后，可使用 `Gst.Pad.add_probe()` 在 OSD 组件的接收衬垫 (Pad) 上安装回调函数。附加后，探针会在衬垫 (Pad) 上发生数据传递时发出通知。创建探针后，可将 `GST_PAD_PROBE_TYPE_BUFFER` 或 `GST_PAD_PROBE_TYPE_BUFFER_LIST` 传递至 `gst_pad_add_probe()`。设计回调函数旨在使用单个缓冲区，因此要使用 `GST_PAD_PROBE_TYPE_BUFFER`。

In [ ]:
# DO NOT CHANGE THIS CELL
# Add probe to nvdsosd plugin's sink
osdsinkpad=nvosd.get_static_pad("sink")
probe_id=osdsinkpad.add_probe(Gst.PadProbeType.BUFFER, osd_sink_pad_buffer_probe)
print('Attached probe')

<a name='s5.6'></a>
### 启动工作流水线 ###
工作流水线中的总线可用于侦听消息。运行 GLib/Gtk+ `Mainloop`（或定期迭代默认的 GLib 主上下文），并使用 `add_signal_watch()` 将侦听程序附加至总线。这样，`GLib.Mainloop` 就会检查总线是否有新消息，并随时通知我们。在缓冲区内容到达 OSD 组件的接收衬垫 (Pad) 时，系统会调用回调函数。然后，将工作流设置为“`PLAYING`”状态。最后，在工作流创建完毕后，将其设置为"`NULL`"状态，以执行清除操作。

In [ ]:
# DO NOT CHANGE THIS CELL
# Create an event loop
loop=GLib.MainLoop()

# Feed GStreamer bus messages to loop
bus=pipeline.get_bus()
bus.add_signal_watch()
bus.connect("message", bus_call, loop)
print('Added bus message handler')

In [ ]:
# DO NOT CHANGE THIS CELL
# Start play back and listen to events - this will generate the output_03_raw.mpeg4 file
print("Starting pipeline")
pipeline.set_state(Gst.State.PLAYING)
try:
    loop.run()
except:
    pass

# Cleaning up as the pipeline comes to an end
pipeline.set_state(Gst.State.NULL)

<a name='s5.7'></a>
### 查看推理 ###
我们使用 [FFmpeg](https://ffmpeg.org/) 工具将视频转换为兼容 JupyterLab 的格式。`ffmpeg` 的通用语法是：
`ffmpeg [global_options] {[input_file_options] -i input_url} ... {[output_file_options] output_url} ...`
* 我们使用 `-loglevel quiet` 标志屏蔽日志

In [ ]:
# DO NOT CHANGE THIS CELL
# Convert MPEG4 video file to MP4 container file
!ffmpeg -i /dli/task/output_03_encoded.mpeg4 /dli/task/output_03.mp4 \
        -y \
        -loglevel quiet

# View the output video
Video("output_03.mp4", width=720)

<p><img src='images/tip.png' width=720></p>

**问题**：运行 DeepStream 工作流水线时出现“找不到输入缓冲区的 NvDsBatchMeta”错误？ 
<br>
**解决方案**：工作流水线中没有 Gst-nvstreammux 插件。从 DeepStream 4.0 开始，Gst-nvstreammux 便是必要插件

**问题**：DeepStream 应用运行速度慢。
<br>
**解决方案**：创建 `Gst-nvstreammux` 插件时，将宽度和高度设置为流的分辨率。

**问题**：DeepStream 应用无法运行。
<br>
**解决方案**：检查 `nvinfer` 的配置文件，尤其要仔细检查 target_classes。类列表必须与训练的网络相匹配。列表的顺序必须与训练期间的列表顺序相同。

**您做得很好！**准备就绪后，我们开始学习[下一个 Notebook](./04_building_a_MDNN_DeepStream_application.ipynb)。

<img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/>